In [ ]:
import os
os.chdir("/home/yhding/Repo/NeuAvatar2/")
os.environ['CUDA_VISIBLE_DEVICES']='10'
from nha.util.render import create_intrinsics_matrix
import torch
from nha.models.nha_optimizer import NHAOptimizer
import numpy as np
import matplotlib.pyplot as plt


from nha.data.real import RealDataModule, tracking_results_2_data_batch
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
from nha.util.general import *
from nha.util.render import create_intrinsics_matrix

from pathlib import Path

import cv2
from pathlib import Path

In [ ]:
# target ckpt
target_ckpt = 'ckpts_and_data/nha/person_0004.ckpt'
target_model = NHAOptimizer.load_from_checkpoint(target_ckpt).cuda().eval()

# both tracking results
driving_tracking_results_path = 'ckpts_and_data/tracking/person_0000.npz'
# driving_tracking_results_path = 'OUTPUT_PATH/tracking_0/tracked_flame_params.npz'
target_tracking_results_path = 'ckpts_and_data/tracking/person_0004.npz'
driving_tracking_results = np.load(driving_tracking_results_path)
target_tracking_results = np.load(target_tracking_results_path)


frameid2folder = lambda x: f"{x:04d}"

In [ ]:
@torch.no_grad()
def frame_reenact(target_model: NHAOptimizer, target_tracking_results: dict,
                   driving_tracking_results: dict, 
                  driving_frame_id=0, target_base_frame_id=0):
    
#     driving_path = Path('out_dir/frame_' + frameid2imagname(driving_frame_id))
    driving_path = Path('ckpts_and_data/data/person_0000/frame_' + frameid2folder(driving_frame_id))

    driver_frame = os.path.join(driving_path, 'image_0000.png')
    rgba_driver = cv2.imread(driver_frame)
    
    base_target_sample = dict_2_device(tracking_results_2_data_batch(target_tracking_results, [target_base_frame_id]),
                                       target_model.device)
    base_target_params = target_model._create_flame_param_batch(base_target_sample)
    base_driving_sample = dict_2_device(tracking_results_2_data_batch(driving_tracking_results, [target_base_frame_id]),
                                       target_model.device)
    base_driving_params = target_model._create_flame_param_batch(base_driving_sample)
    
    batch = dict_2_device(tracking_results_2_data_batch(driving_tracking_results, [driving_frame_id]), target_model.device)
    batch["cam_intrinsic"] = base_target_sample["cam_intrinsic"].expand_as(batch["cam_intrinsic"])
    batch["cam_extrinsic"] = base_target_sample["cam_extrinsic"].expand_as(batch["cam_extrinsic"])

    rgba_target = target_model.predict_reenaction(batch, driving_model=target_model,
                                                     base_target_params=base_target_params,
                                                     base_driving_params=base_driving_params)        
    
    return rgba_driver, rgba_target
    

In [ ]:
rgba_driver, rgba_target = frame_reenact(target_model, target_tracking_results, 
                                         driving_tracking_results, 
                                         driving_frame_id=30)
## show range 
fig, axes = plt.subplots(ncols=2, figsize=(20,10))
axes[0].imshow(cv2.cvtColor(rgba_driver, cv2.COLOR_BGR2RGB))
axes[1].imshow(rgba_target[0,:3].cpu().permute(1,2,0))

[a.axis("off") for a in axes]
plt.show()
    